In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter
import matplotlib as mpl

import numpy as np
import time
#from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io
from scipy.io import savemat

from smt.sampling_methods import LHS

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())
    

cuda:0


In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
# %cd '/content/gdrive/MyDrive/Virginia Tech /Fall 2022/Codes from GPU/PINN_Stan/2D Klein Gordon/stan'

In [4]:
# !pip install smt

In [5]:
lr_tune = np.array([0.05,0.1,0.25,0.5,1])

In [6]:
def true_2D_1(xt): #True function for 2D_1 Klein Gordon Equation x \in [-50,50] , t \in [0,10]
    y = xt[:,0]*np.cos(xt[:,1])
    return y.reshape(-1,1)    

In [7]:

loss_thresh = 0.01

x = np.linspace(-5,5,500).reshape(-1,1)
t = np.linspace(0,10,1000).reshape(-1,1)

X,T = np.meshgrid(x,t)

X = X.flatten('F').reshape(-1,1)
T = T.flatten('F').reshape(-1,1)
  
xt = np.hstack((X,T))

y_true = true_2D_1(xt)
y_true_norm = np.linalg.norm(y_true,2)

#bound_pts_idx = ((X == -5) + (X == 5) + (T == 0)).reshape(-1,)

#xt_bound = xt[bound_pts_idx,:]
#y_bound = y_true[bound_pts_idx,:]


xt_test_tensor = torch.from_numpy(xt).float().to(device)


lb_xt = xt[0]
ub_xt = xt[-1]

In [8]:
def trainingdata(N_I,N_B,N_f,seed):
    '''Boundary Conditions''' 
    
    np.random.seed(seed)
    x_BC1 = np.random.uniform(size = N_I).reshape(-1,1)
    t_BC1 = np.zeros((N_I,1))
    samples = np.hstack((x_BC1,t_BC1))
    xt_BC1 = lb_xt + (ub_xt - lb_xt)*samples
    y_BC1 = true_2D_1(xt_BC1)
    
    x_BC2 = np.zeros((int(N_B/2),1))
    t_BC2 = np.random.uniform(size = int(N_B/2)).reshape(-1,1)
    samples = np.hstack((x_BC2,t_BC2))
    xt_BC2 = lb_xt + (ub_xt - lb_xt)*samples
    y_BC2 = true_2D_1(xt_BC2)
    
    x_BC3 = np.ones((int(N_B/2),1))
    t_BC3 = np.random.uniform(size = int(N_B/2)).reshape(-1,1)
    samples = np.hstack((x_BC3,t_BC3))
    xt_BC3 = lb_xt + (ub_xt - lb_xt)*samples
    y_BC3 = true_2D_1(xt_BC3)

    xt_BC = np.vstack((xt_BC1,xt_BC2,xt_BC3))
    y_BC = np.vstack((y_BC1,y_BC2,y_BC3))

    '''Collocation Points'''

    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,t)
    x01 = np.array([[0.0,1.0],[0.0,1.0]])
    sampling = LHS(xlimits=x01,random_state =seed)
    samples = sampling(N_f)
    xt_coll = lb_xt + (ub_xt - lb_xt)*samples
    
    xt_coll = np.vstack((xt_coll, xt_BC)) # append training points to collocation points 

    return xt_coll, xt_BC, y_BC

In [9]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers):
        super().__init__() #call __init__ from parent class 
              
        'activation function'
        self.activation = nn.Tanh()

        'loss function'
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        
        # std = gain * sqrt(2/(input_dim+output_dim))
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)
            
        

            
    'foward pass'
    def forward(self,xt):
        if torch.is_tensor(xt) != True:         
            xt = torch.from_numpy(xt)                
        
        ubxt = torch.from_numpy(ub_xt).float().to(device)
        lbxt = torch.from_numpy(lb_xt).float().to(device)
    
                      
        #preprocessing input 
        xt = (xt - lbxt)/(ubxt - lbxt)
        
        #convert to float
        a = xt.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            a = self.activation(z)
            
        a = self.linears[-1](a) 
         
        return a
                        
    def loss_BC(self,xt,y):
                
        loss_bc = self.loss_function(self.forward(xt), y)
                
        return loss_bc
    
    def loss_PDE(self, xt_coll, f_hat):
        
        g = xt_coll.clone()             
        g.requires_grad = True
        y = self.forward(g) 
        
        
        y_x_t = autograd.grad(y,g,torch.ones([xt_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        y_xx_tt = autograd.grad(y_x_t,g,torch.ones(xt_coll.shape).to(device), create_graph=True,allow_unused = True)[0]

        #du_dx = u_x_t[:,[0]]
        
        d2y_dx2 = y_xx_tt[:,[0]]
        d2y_dt2 = y_xx_tt[:,[1]]    
        

        f = d2y_dt2 - d2y_dx2 + torch.pow(y,2) + (g[:,0]*torch.cos(g[:,1])).reshape(-1,1) - (torch.pow(g[:,0],2)*torch.pow(torch.cos(g[:,1]),2)).reshape(-1,1)
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    def loss(self,xt_BC,y_BC,xt_coll,f_hat):

        loss_BC = self.loss_BC(xt_BC,y_BC)
        loss_f = self.loss_PDE(xt_coll,f_hat)
        
        loss_val = loss_BC + loss_f
        
        return loss_val
         
    'test neural network'
    def test(self):
        y_pred = self.forward(xt_test_tensor)
        y_pred = y_pred.cpu().detach().numpy()
   
        return y_pred
    
    def test_loss(self):
        y_pred = self.test()
        
        test_mse = np.mean(np.square(y_pred.reshape(-1,1) - y_true.reshape(-1,1)))
        test_re = np.linalg.norm(y_pred.reshape(-1,1) - y_true.reshape(-1,1),2)/y_true_norm
        
        return test_mse, test_re  

In [10]:
def data_update(loss_np):
    train_loss.append(loss_np)
    
    test_mse, test_re = PINN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [11]:
def train_step(xt_BC, y_BC, xt_coll, f_hat,seed):
    # x_coll_np_array = colloc_pts(N_f,seed*123)
    # x_coll_train = torch.from_numpy(x_coll_np_array).float().to(device)        
    
    # f_hat = torch.zeros(x_coll_train.shape[0],1).to(device)
    
    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(xt_BC, y_BC, xt_coll,f_hat)
        loss.backward()
        #print(loss.cpu().detach().numpy())
        return loss

    optimizer.step(closure)

In [12]:
def train_model(max_iter,rep): 
  print(rep) 
  torch.manual_seed(rep*11)
  start_time = time.time() 
  thresh_flag = 0

  xt_coll, xt_BC, y_BC = trainingdata(N_I,N_B,N_f,rep*11)
  xt_coll = torch.from_numpy(xt_coll).float().to(device)
  xt_BC = torch.from_numpy(xt_BC).float().to(device)
  y_BC = torch.from_numpy(y_BC).float().to(device)

  f_hat = torch.zeros(xt_coll.shape[0],1).to(device)

  for i in range(max_iter):
    train_step(xt_BC, y_BC, xt_coll,f_hat,i)
    
    loss_np = PINN.loss(xt_BC, y_BC, xt_coll,f_hat).cpu().detach().numpy()
    if(thresh_flag == 0):
        if(loss_np < loss_thresh):
            time_threshold[rep] = time.time() - start_time
            epoch_threshold[rep] = i+1            
            thresh_flag = 1       
    data_update(loss_np)
    print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1])

  elapsed_time[rep] = time.time() - start_time  
  print('Training time: %.2f' % (elapsed_time[rep]))

In [13]:
for tune_reps in range(5):
#for tune_reps in range(1,5):
  max_reps = 10 #10
  max_iter = 100 #100
  label = "KG_tanh_tune"+str(tune_reps)

  train_loss_full = []
  test_mse_full = []
  test_re_full = []

  elapsed_time= np.zeros((max_reps,1))
  time_threshold = np.empty((max_reps,1))
  time_threshold[:] = np.nan
  epoch_threshold = max_iter*np.ones((max_reps,1))

  N_I = 200  #Total number of data points for 'y'
  N_B = 400
  N_f = 10000 #Total number of collocation points

  for reps in range(max_reps):
      print(reps)
      print(label)

      train_loss = []
      test_mse_loss = []
      test_re_loss = []

      
      torch.manual_seed(reps*36)

      layers = np.array([2,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers

      PINN = Sequentialmodel(layers)
    
      PINN.to(device)

      'Neural Network Summary'
      print(PINN)

      params = list(PINN.parameters())
      

      optimizer = torch.optim.LBFGS(PINN.parameters(), lr=lr_tune[tune_reps], 
                                max_iter = 10, 
                                max_eval = 15, 
                                tolerance_grad = 1e-8, 
                                tolerance_change = 1e-8, 
                                history_size = 100, 
                                line_search_fn = 'strong_wolfe')



          
      train_model(max_iter,reps)
      
      torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
      train_loss_full.append(train_loss)
      test_mse_full.append(test_mse_loss)
      test_re_full.append(test_re_loss)
      #elapsed_time[reps] = time.time() - start_time
    


      #print('Training time: %.2f' % (elapsed_time[reps]))
      
  mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full,"test_re_loss": test_re_full,"Time": elapsed_time, "label": label}
  savemat(label+'.mat', mdic)

0
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 58.941574 Test MSE 8.656132292244305 Test RE 1.4062741797472664
1 Train Loss 58.2084 Test MSE 8.630431149985787 Test RE 1.4041849255864758
2 Train Loss 56.60464 Test MSE 8.214433591342935 Test RE 1.3699252539228532
3 Train Loss 49.17374 Test MSE 7.920135555490482 Test RE 1.34516131047

96 Train Loss 0.02024583 Test MSE 0.005688014573864776 Test RE 0.03604860063037551
97 Train Loss 0.01865324 Test MSE 0.005106788495484318 Test RE 0.03415717902401208
98 Train Loss 0.016006708 Test MSE 0.004737985862668008 Test RE 0.03290068502930461
99 Train Loss 0.015055071 Test MSE 0.004361546619424858 Test RE 0.03156663726156854
Training time: 68.43
1
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): L

93 Train Loss 1.446276 Test MSE 2.4748358253842935 Test RE 0.751936657210645
94 Train Loss 1.3869934 Test MSE 2.5147236454967428 Test RE 0.7579720526836445
95 Train Loss 1.3547534 Test MSE 2.5238785979514016 Test RE 0.7593505130920775
96 Train Loss 1.3218989 Test MSE 2.4907521052341832 Test RE 0.7543507271514012
97 Train Loss 1.2923701 Test MSE 2.5147432726746715 Test RE 0.7579750106276205
98 Train Loss 1.2770398 Test MSE 2.5114420255621392 Test RE 0.757477328697679
99 Train Loss 1.2591393 Test MSE 2.4979669457220552 Test RE 0.755442482646446
Training time: 70.03
2
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out

90 Train Loss 6.614764 Test MSE 4.006848690466505 Test RE 0.9567743669746352
91 Train Loss 6.482505 Test MSE 3.9937085055069868 Test RE 0.9552042407402054
92 Train Loss 6.397782 Test MSE 3.916393712056599 Test RE 0.9459130836908413
93 Train Loss 6.3522215 Test MSE 3.878658434238948 Test RE 0.9413450176493083
94 Train Loss 6.2789893 Test MSE 3.7828916383700997 Test RE 0.9296511498479264
95 Train Loss 6.1801577 Test MSE 3.7050465484179878 Test RE 0.9200361555857076
96 Train Loss 6.1143394 Test MSE 3.745009574943728 Test RE 0.9249846515860521
97 Train Loss 6.0688505 Test MSE 3.810761644019415 Test RE 0.9330694128036687
98 Train Loss 6.033643 Test MSE 3.764924981606173 Test RE 0.9274408562228691
99 Train Loss 5.9334207 Test MSE 3.7313809614088727 Test RE 0.9233000433106823
Training time: 66.50
3
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features

87 Train Loss 7.0920234 Test MSE 6.616724651011715 Test RE 1.2295029961646435
88 Train Loss 7.0343084 Test MSE 6.630128484806205 Test RE 1.230747699322357
89 Train Loss 6.977806 Test MSE 6.622804796091974 Test RE 1.2300677650226082
90 Train Loss 6.8590117 Test MSE 6.560157830710828 Test RE 1.224236163823656
91 Train Loss 6.8241134 Test MSE 6.554374892250807 Test RE 1.2236964478980088
92 Train Loss 6.778221 Test MSE 6.571684769414808 Test RE 1.225311252416257
93 Train Loss 6.7261624 Test MSE 6.593210897000526 Test RE 1.2273164187615315
94 Train Loss 6.6737285 Test MSE 6.640099794324609 Test RE 1.2316728364140335
95 Train Loss 6.6324396 Test MSE 6.665302916374518 Test RE 1.2340080875280248
96 Train Loss 6.5890293 Test MSE 6.652517728828356 Test RE 1.2328240004691577
97 Train Loss 6.53466 Test MSE 6.6663427837625875 Test RE 1.2341043438225991
98 Train Loss 6.505369 Test MSE 6.651122110310735 Test RE 1.2326946778336216
99 Train Loss 6.4693327 Test MSE 6.6602811619810325 Test RE 1.233543138

...

0 Train Loss 57.84659 Test MSE 8.624517827165134 Test RE 1.4037037896258013
1 Train Loss 56.25527 Test MSE 8.884038324354085 Test RE 1.4246666957012264
2 Train Loss 51.232338 Test MSE 9.051817073813327 Test RE 1.4380564842603885
3 Train Loss 47.036163 Test MSE 8.518165990108814 Test RE 1.3950221702682097
4 Train Loss 46.095253 Test MSE 8.589276113679123 Test RE 1.400832929679729
5 Train Loss 45.461067 Test MSE 8.565185593454437 Test RE 1.3988670777615169
6 Train Loss 44.969505 Test MSE 8.546325498161568 Test RE 1.397326112566777
7 Train Loss 44.63409 Test MSE 8.577711585743788 Test RE 1.399889577301526
8 Train Loss 44.324615 Test MSE 8.445491427625361 Test RE 1.3890584572275244
9 Train Loss 44.27434 Test MSE 8.433905009901492 Test RE 1.3881053016065037
10 Train Loss 43.912197 Test MSE 8.283436416300129 Test RE 1.3756670391617112
11 Train Loss 43.4767 Test MSE 7.980296728674815 Test RE 1.3502605530317082
12 Train Loss 41.524086 Test MSE 7.656169391474393 Test RE 1.3225552598732924
13 Tr

0 Train Loss 58.922356 Test MSE 8.592539603191742 Test RE 1.401099027157125
1 Train Loss 58.36521 Test MSE 8.771734757476878 Test RE 1.4156334163731543
2 Train Loss 56.24039 Test MSE 9.024334023473816 Test RE 1.4358717171976065
3 Train Loss 50.224056 Test MSE 8.502837397017686 Test RE 1.3937664213199001
4 Train Loss 47.0163 Test MSE 8.60660822566189 Test RE 1.4022455724341065
5 Train Loss 46.107265 Test MSE 8.539004943876112 Test RE 1.3967275282326284
6 Train Loss 45.910934 Test MSE 8.38365717197409 Test RE 1.3839640710904033
7 Train Loss 45.69084 Test MSE 8.43454868356527 Test RE 1.3881582705323519
8 Train Loss 44.566643 Test MSE 8.864547246613323 Test RE 1.423103018488749
9 Train Loss 41.309044 Test MSE 7.953788658037985 Test RE 1.34801611426264
10 Train Loss 40.456245 Test MSE 7.679960455242915 Test RE 1.3246085445480036
11 Train Loss 39.845108 Test MSE 7.818849341140234 Test RE 1.3365323736246006
12 Train Loss 39.70217 Test MSE 7.807160360600896 Test RE 1.3355329590383147
13 Train 

0 Train Loss 55.650574 Test MSE 8.60197556745681 Test RE 1.4018681299686788
1 Train Loss 54.86461 Test MSE 8.402744866795178 Test RE 1.3855386622370254
2 Train Loss 51.887787 Test MSE 8.849310193759786 Test RE 1.4218794240229917
3 Train Loss 46.955612 Test MSE 8.191739447009226 Test RE 1.3680315881261211
4 Train Loss 45.37139 Test MSE 7.967339220356944 Test RE 1.34916390708431
5 Train Loss 43.1849 Test MSE 8.333304249215594 Test RE 1.379801711761817
6 Train Loss 42.437057 Test MSE 7.96627696513363 Test RE 1.3490739646242063
7 Train Loss 41.601345 Test MSE 7.900671542821379 Test RE 1.3435074030184182
8 Train Loss 41.048977 Test MSE 8.06482256891995 Test RE 1.3573925736566514
9 Train Loss 40.131573 Test MSE 8.100228055267076 Test RE 1.3603688643942367
10 Train Loss 39.670906 Test MSE 7.971599832043073 Test RE 1.3495245980923494
11 Train Loss 39.45143 Test MSE 8.012158416743068 Test RE 1.3529533554506439
12 Train Loss 39.237537 Test MSE 7.934604585097747 Test RE 1.3463894648208719
13 Trai

0 Train Loss 57.000656 Test MSE 8.445810603014412 Test RE 1.3890847049058592
1 Train Loss 55.939545 Test MSE 8.394108009694097 Test RE 1.3848264083109578
2 Train Loss 50.878437 Test MSE 7.677197914425317 Test RE 1.3243702871819383
3 Train Loss 44.552315 Test MSE 7.572762073292156 Test RE 1.3153314861753056
4 Train Loss 39.80375 Test MSE 7.993292684717419 Test RE 1.3513595590572796
5 Train Loss 37.758316 Test MSE 7.521643582843143 Test RE 1.3108845213236027
6 Train Loss 36.01625 Test MSE 7.100889158504302 Test RE 1.2736920176855613
7 Train Loss 34.45152 Test MSE 6.860776124247468 Test RE 1.2519721964936554
8 Train Loss 34.09686 Test MSE 6.935014664394547 Test RE 1.2587275916449907
9 Train Loss 32.0185 Test MSE 6.393913118580349 Test RE 1.2086245892330305
10 Train Loss 30.738543 Test MSE 6.466002465950784 Test RE 1.2154189218402685
11 Train Loss 29.378317 Test MSE 6.466589817696103 Test RE 1.2154741230420851
12 Train Loss 27.335304 Test MSE 5.891758448514112 Test RE 1.1601937642168352
13

0 Train Loss 56.398453 Test MSE 8.600345953614404 Test RE 1.40173533419985
1 Train Loss 52.33442 Test MSE 7.297710573894817 Test RE 1.291223370425315
2 Train Loss 44.100212 Test MSE 8.261962858855142 Test RE 1.3738827775727669
3 Train Loss 43.60495 Test MSE 8.501815653881025 Test RE 1.3936826778592155
4 Train Loss 43.391846 Test MSE 8.495136232603443 Test RE 1.3931350993639384
5 Train Loss 43.344067 Test MSE 8.481502565166648 Test RE 1.3920167437057855
6 Train Loss 43.304886 Test MSE 8.469997517869153 Test RE 1.3910722970616036
7 Train Loss 43.285362 Test MSE 8.485414450302839 Test RE 1.3923377234980765
8 Train Loss 43.062073 Test MSE 8.454727895497413 Test RE 1.3898178261723282
9 Train Loss 42.709045 Test MSE 8.444305325157849 Test RE 1.3889609126661038
10 Train Loss 42.164543 Test MSE 8.363600943004489 Test RE 1.38230765072127
11 Train Loss 41.238625 Test MSE 8.369976233195997 Test RE 1.382834393551182
12 Train Loss 40.17991 Test MSE 7.986774073090145 Test RE 1.350808422925437
13 Tra

In [14]:
import scipy.io as sio

In [15]:
for tune_reps in range(5):
    label = "KG_tanh_tune"+str(tune_reps)+".mat"
    data = sio.loadmat(label)
    re = np.array(data["test_re_loss"])
    print(np.mean(re[:,-1]))

0.737439545138477
0.6045129454552253
0.5595302907178141
0.47469131344146553
0.45481459073362557


In [16]:
label

'KG_tanh_tune4.mat'